In [37]:
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import graphlab

In [38]:
titanic_df = pd.read_csv("train.csv", dtype={"Age": np.float64}, )
test_df    = pd.read_csv("test.csv", dtype={"Age": np.float64}, )

train=graphlab.SFrame.read_csv('train.csv')
titanic_df,test_df = train.random_split(.8,seed=1)
testing=test_df
titanic_df=titanic_df.to_dataframe()
test_df=test_df.to_dataframe()
test_df.drop("Survived", axis=1, inplace=True)

In [39]:
titanic_df.info()
print "---"
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 90.5+ KB
---
<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null f

In [40]:
#pclass column

pclass_dummies_titanic=pd.get_dummies(titanic_df.Pclass)
pclass_dummies_titanic = pclass_dummies_titanic.rename(columns={1: 'PClass=1',2:'PClass=2',3:'PClass=3'})
pclass_dummies_test=pd.get_dummies(test_df.Pclass)
pclass_dummies_test = pclass_dummies_test.rename(columns={1: 'PClass=1',2:'PClass=2',3:'PClass=3'})
titanic_df=titanic_df.join(pclass_dummies_titanic)
test_df=test_df.join(pclass_dummies_test)

titanic_df.drop(['Pclass'], axis=1, inplace=True)
test_df.drop(['Pclass'], axis=1, inplace=True)

In [41]:
titanic_df.head()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,PClass=1,PClass=2,PClass=3
0,1,0,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,1,0,0
2,3,1,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,1,0,0
4,5,0,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,0,0,1


In [42]:
#sex column
sex_dummies_titanic=pd.get_dummies(titanic_df.Sex)
sex_dummies_test=pd.get_dummies(test_df.Sex)
titanic_df.join(sex_dummies_titanic)
test_df.join(sex_dummies_test)
titanic_df.drop(['Sex'],axis=1, inplace=True)
test_df.drop(['Sex'],axis=1, inplace=True)

Age column
titanic_df_survived=titanic_df[titanic_df.Survived==1]
titanic_df_unsurvived=titanic_df[titanic_df.Survived==0]
titanic_df_survived_fareMore26=titanic_df_survived[titanic_df_survived.Fare>26]
mean_titanic_df_survived_fareMore26_age=32.16
sd_titanic_df_survived_fareMore26_age=15.27
count_nan = titanic_df_survived_fareMore26["Age"].isnull().sum()
rand_1 = np.random.normal(mean_titanic_df_survived_fareMore26_age,sd_titanic_df_survived_fareMore26_age,count_nan)
titanic_df_survived_fareMore26["Age"][np.isnan(titanic_df_survived_fareMore26["Age"])] = rand_1


In [43]:
#Age column
#filling the na values of survived passengers and fare>26
mean_titanic_df_survived_fareMore26_age=32.16
sd_titanic_df_survived_fareMore26_age=15.27
count_nan = len(titanic_df["Age"][titanic_df.Fare.apply(lambda x: x>26)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==1)])
rand = np.random.normal(mean_titanic_df_survived_fareMore26_age,sd_titanic_df_survived_fareMore26_age,count_nan)
titanic_df["Age"][titanic_df.Fare.apply(lambda x: x>26)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==1)]=rand

#filling the na values of survived passengers and fare<26
mean_titanic_df_survived_fareLess26_age=24.366
sd_titanic_df_survived_fareLess26_age=13.562
count_nan = len(titanic_df["Age"][titanic_df.Fare.apply(lambda x: x<=26)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==1)])
rand = np.random.normal(mean_titanic_df_survived_fareLess26_age,sd_titanic_df_survived_fareLess26_age,count_nan)
titanic_df["Age"][titanic_df.Fare.apply(lambda x: x<26)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==1)]=rand

#filling the na values of unsurvived passengers and fare>10.5
mean_titanic_df_unsurvived_fareMore10_age=31.89
sd_titanic_df_unsurvived_fareMore10_age=16.23
count_nan = len(titanic_df["Age"][titanic_df.Fare.apply(lambda x: x>10.5)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==0)])
rand = np.random.normal(mean_titanic_df_unsurvived_fareMore10_age,sd_titanic_df_unsurvived_fareMore10_age,count_nan)
titanic_df["Age"][titanic_df.Fare.apply(lambda x: x>10.5)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==0)]=rand

#filling the na values of unsurvived passengers and fare<=10.5
mean_titanic_df_unsurvived_fareLess10_age=29.30
sd_titanic_df_unsurvived_fareLess10_age=16.23
count_nan = len(titanic_df["Age"][titanic_df.Fare.apply(lambda x: x<=10.5)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==0)])
rand = np.random.normal(mean_titanic_df_unsurvived_fareLess10_age,sd_titanic_df_unsurvived_fareLess10_age,count_nan)
titanic_df["Age"][titanic_df.Fare.apply(lambda x: x<10.5)&np.isnan(titanic_df["Age"])&(titanic_df.Survived==0)]=rand


C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-package

In [44]:
#filling out na values of age in test

mean_titanic_df_survived_fareMore26_age=32.16
sd_titanic_df_survived_fareMore26_age=15.27
count_nan = len(test_df["Age"][test_df.Fare.apply(lambda x: x>26)&np.isnan(test_df["Age"])])
rand = np.random.normal(mean_titanic_df_survived_fareMore26_age,sd_titanic_df_survived_fareMore26_age,count_nan)
test_df["Age"][test_df.Fare.apply(lambda x: x>26)&np.isnan(test_df["Age"])]=rand

#filling the na values of test passengers and fare<26
mean_titanic_df_survived_fareLess26_age=24.366
sd_titanic_df_survived_fareLess26_age=13.562
count_nan = len(test_df["Age"][test_df.Fare.apply(lambda x: x<=26)&np.isnan(test_df["Age"])])
rand = np.random.normal(mean_titanic_df_survived_fareLess26_age,sd_titanic_df_survived_fareLess26_age,count_nan)
test_df["Age"][test_df.Fare.apply(lambda x: x<=26)&np.isnan(test_df["Age"])]=rand

#filling the na values of test passengers and fare>10.5
mean_titanic_df_unsurvived_fareMore10_age=31.89
sd_titanic_df_unsurvived_fareMore10_age=16.23
count_nan = len(test_df["Age"][test_df.Fare.apply(lambda x: x>10.5)&np.isnan(test_df["Age"])])
rand = np.random.normal(mean_titanic_df_unsurvived_fareMore10_age,sd_titanic_df_unsurvived_fareMore10_age,count_nan)
test_df["Age"][test_df.Fare.apply(lambda x: x>10.5)&np.isnan(test_df["Age"])]=rand

#filling the na values of unsurvived passengers and fare<=10.5
mean_titanic_df_unsurvived_fareLess10_age=29.30
sd_titanic_df_unsurvived_fareLess10_age=16.23
count_nan = len(test_df["Age"][test_df.Fare.apply(lambda x: x<=10.5)&np.isnan(test_df["Age"])])
rand = np.random.normal(mean_titanic_df_unsurvived_fareLess10_age,sd_titanic_df_unsurvived_fareLess10_age,count_nan)
test_df["Age"][test_df.Fare.apply(lambda x: x<10.5)&np.isnan(test_df["Age"])]=rand


C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-packages\IPython\kernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ATX\AppData\Local\Dato\Dato Launcher\lib\site-package

In [45]:
#drop ticket column
titanic_df.drop(['Ticket'], axis=1, inplace=True)
test_df.drop(['Ticket'], axis=1, inplace=True)

In [46]:
#drop cabin column
titanic_df.drop(['Cabin'], axis=1, inplace=True)
test_df.drop(['Cabin'], axis=1, inplace=True)

In [47]:
#embarked column
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 10 columns):
PassengerId    418 non-null int64
Name           418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           417 non-null float64
Embarked       418 non-null object
PClass=1       418 non-null float64
PClass=2       418 non-null float64
PClass=3       418 non-null float64
dtypes: float64(5), int64(3), object(2)
memory usage: 35.9+ KB


In [48]:
#null fare 
test_df[pd.isnull(test_df).any(axis=1)]

,PassengerId,Name,Age,SibSp,Parch,Fare,Embarked,PClass=1,PClass=2,PClass=3
152,1044,"Storey, Mr. Thomas",60.5,0,0,NaN,S,0,0,1


In [49]:
test_df["Fare"].fillna(28.28, inplace=True)#median fare over age 60

In [50]:
#NAN embarked column
titanic_df[pd.isnull(titanic_df).any(axis=1)]

,PassengerId,Survived,Name,Age,SibSp,Parch,Fare,Embarked,PClass=1,PClass=2,PClass=3
61,62,1,"Icard, Miss. Amelie",38,0,0,80,NaN,1,0,0
829,830,1,"Stone, Mrs. George Nelson (Martha Evelyn)",62,0,0,80,NaN,1,0,0


In [51]:
titanic_df["Embarked"] = titanic_df["Embarked"].fillna("S")

In [52]:

embarked_dummies_titanic=pd.get_dummies(titanic_df.Embarked)
embarked_dummies_test=pd.get_dummies(test_df.Embarked)
embarked_dummies_titanic.drop(['S'], axis=1, inplace=True)
embarked_dummies_test.drop(['S'], axis=1, inplace=True)
titanic_df=titanic_df.join(embarked_dummies_titanic)
test_df=test_df.join(embarked_dummies_test)

titanic_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

In [53]:

def comparefun(x,y1,y2):
    if x<y1:
        if x>=y2:
            return 1
    return 0

#Manipulating Fare Column
titanic_df['fare<7.91']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(7.91),np.float64(0)))
test_df['fare<7.91']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(7.91),np.float64(0)))
titanic_df['7.91<fare<14.45']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(14.45),np.float64(7.91)))
test_df['7.91<fare<14.45']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(14.45),np.float64(7.91)))
titanic_df['14.45<fare<31']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(31),np.float64(14.45)))
test_df['14.45<fare<31']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(31),np.float64(14.45)))
titanic_df['fare>31']=titanic_df.Fare.apply(lambda x: comparefun(x,np.float64(1000),np.float64(31)))
test_df['fare>31']=test_df.Fare.apply(lambda x: comparefun(x,np.float64(10000),np.float64(31)))

titanic_df.drop(['Fare'], axis=1, inplace=True)
test_df.drop(['Fare'], axis=1, inplace=True)


In [54]:
#manipulating Sibsp values

def equalComparison(x,y):
    if x==y:
        return 1
    else: 
        return 0
def equalGreaterComparison(x,y,z):
    if x<y:
        if x>=z:
            return 1
    return 0
"""    
for i in range(8):
    colName="Sibsp="+str(i)
    titanic_df[colName]=titanic_df.SibSp.apply(lambda x: equalComparison(x,np.float64(i)))
    test_df[colName]=test_df.SibSp.apply(lambda x: equalComparison(x,np.float64(i)))
"""

colName="Sibsp=0"
titanic_df[colName]=titanic_df.SibSp.apply(lambda x: equalComparison(x,np.float64(0)))
test_df[colName]=test_df.SibSp.apply(lambda x: equalComparison(x,np.float64(0)))

colName="Sibsp=1"
titanic_df[colName]=titanic_df.SibSp.apply(lambda x: equalComparison(x,np.float64(1)))
test_df[colName]=test_df.SibSp.apply(lambda x: equalComparison(x,np.float64(1)))

colName="Sibsp=2"
titanic_df[colName]=titanic_df.SibSp.apply(lambda x: equalComparison(x,np.float64(2)))
test_df[colName]=test_df.SibSp.apply(lambda x: equalComparison(x,np.float64(2)))

#colName="Sibsp<=2"
#titanic_df[colName]=titanic_df.SibSp.apply(lambda x: equalGreaterComparison(x,np.float64(3),np.float64(0)))
#test_df[colName]=test_df.SibSp.apply(lambda x: equalGreaterComparison(x,np.float64(3),np.float64(0)))

colName="Sibsp>2"
titanic_df[colName]=titanic_df.SibSp.apply(lambda x: equalGreaterComparison(x,np.float64(100),np.float64(3)))
test_df[colName]=test_df.SibSp.apply(lambda x: equalGreaterComparison(x,np.float64(100),np.float64(3)))

#drop Sibsp columns
titanic_df.drop('SibSp',axis=1, inplace=True)
test_df.drop('SibSp',axis=1, inplace=True)


In [55]:
#manipulating age columns

titanic_df['age<11']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(11),np.float64(0)))
test_df['age<11']=test_df.Age.apply(lambda x: comparefun(x,np.float64(11),np.float64(0)))
titanic_df['11<age<20']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(20),np.float64(0)))
test_df['11<age<20']=test_df.Age.apply(lambda x: comparefun(x,np.float64(20),np.float64(0)))
titanic_df['20<age<31']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(1000),np.float64(20)))
test_df['20<age<31']=test_df.Age.apply(lambda x: comparefun(x,np.float64(1000),np.float64(20)))
titanic_df['age>31']=titanic_df.Age.apply(lambda x: comparefun(x,np.float64(1000),np.float64(31)))
test_df['age>31']=test_df.Age.apply(lambda x: comparefun(x,np.float64(10000),np.float64(31)))

titanic_df.drop(['Age'], axis=1, inplace=True)
test_df.drop(['Age'], axis=1, inplace=True)


In [56]:
traincolumn=titanic_df.columns[3:]
print traincolumn
traincolumn=traincolumn[1:4]+traincolumn[6:10]
print traincolumn

Index([u'Parch', u'PClass=1', u'PClass=2', u'PClass=3', u'C', u'Q',
       u'fare<7.91', u'7.91<fare<14.45', u'14.45<fare<31', u'fare>31',
       u'Sibsp=0', u'Sibsp=1', u'Sibsp=2', u'Sibsp>2', u'age<11', u'11<age<20',
       u'20<age<31', u'age>31'],
      dtype='object')
Index([u'14.45<fare<31', u'7.91<fare<14.45', u'PClass=1', u'PClass=2',
       u'PClass=3', u'fare<7.91', u'fare>31'],
      dtype='object')


In [57]:
y_train=titanic_df.Survived
x_train=titanic_df[titanic_df.columns[3:]]

def conversion_y_train(x):
    if x==0:
        return -1
    return x

y_train=y_train.apply(lambda x: conversion_y_train(x))

In [58]:
x_matrix=np.matrix(x_train)
y_matrix=np.matrix(y_train).transpose()

In [59]:
kernel=np.dot(x_matrix,x_matrix.transpose())

In [60]:
m=x_matrix.shape[0]#length of the dataset
n=x_matrix.shape[1]#number of features
alpha=np.array([0]*m)
b=0
E=np.array([0]*m)
C=np.float64(.001) #regularized parameter
eta=np.float64(0)
passes=0
maxpasses=2
L=0
H=0
tol=1e-5

In [61]:
#calculating f(x(i))
part1=np.dot(alpha,y_matrix)
part2=sum(np.dot(part1[0,0],kernel[:,0]))
part3=b+part2[0,0]
E[0]=part3
kernel[1,2]

1.0

In [27]:
"""

while passes<maxpasses:
    num_changed_alphas=0
    for i in range (m):
        #calculating f(x(i))
        part1=np.dot(alpha,y_matrix)
        part2=sum(np.dot(part1[0,0],kernel[:,i]))
        f_x_i=b+part2[0,0]
        E[i]=f_x_i-y_matrix[i,0]
        if (((y_matrix[i,0]*E[i])<-tol) & alpha[i]<C) | (((y_matrix[i,0]*E[i])>tol) & alpha[i]>0):
            j=np.random.random_integers(0,890)
            while(j==i):
                j=np.random.random_integers(0,890)
                
            #computing f(x(j))
            part1=np.dot(alpha,y_matrix)
            part2=sum(np.dot(part1[0,0],kernel[:,j]))
            f_x_j=b+part2[0,0]
            E[j]=f_x_j-y_matrix[j,0]
            
            #saving old alphas
            alpha_i_old=alpha[i]
            alpha_j_old=alpha[j]
            
            #computing L & H
            
            if(y_matrix[i,0]==y_matrix[j,0]):
                L=max(0,alpha[j]-alpha[i])
                H=min(C,C+alpha[j]-alpha[i])
            else:
                L=max(0,alpha[j]+alpha[i]-C)
                H=min(C,alpha[j]+alpha[i])
            
            if (L==H):
                continue
            
            eta=2*kernel[i,j]-kernel[i,i]-kernel[j,j]
            
            if(eta>=0):
                continue
            alpha[j]=alpha[j]- ((y_matrix[j,0] *(E[i]*E[j]))/eta)
            
            if (alpha[j]>H):
                alpha[j]=H
            elif (alpha[j]>=L & alpha[j]<=H):
                alpha[j]=alpha[j]
            elif (alpha[j]<L):
                alpha[j]=L
                
            if abs(alpha[j]-alpha_j_old)<tol:
                alpha[j]=alpha_j_old
                continue
            
            alpha[i]=alpha[i]+ y_matrix[i,0]*y_matrix[j,0]*(alpha_j_old-alpha[j])
            
            b1= b - E[i] - y_matrix[i,0]*(alpha[i]-alpha_i_old)*kernel[i,i] - y_matrix[j,0]*(alpha[j]-alpha_j_old)*kernel[i,j]
            b2= b - E[j] - y_matrix[i,0]*(alpha[i]-alpha_i_old)*kernel[i,j] - y_matrix[j,0]*(alpha[j]-alpha_j_old)*kernel[j,j]
            
            if (alpha[i]>0 & alpha[i]<C):
                b=b1
            elif (alpha[j]>0 & alpha[j]<C):
                b=b2
            else:
                b=(b1+b2)/2
                
            num_changed_alphas=num_changed_alphas+1
            
        if num_changed_alphas==0:
            passes=passes+1
        else:
            passes=0
        
"""
    

'\n\nwhile passes<maxpasses:\n    num_changed_alphas=0\n    for i in range (m):\n        #calculating f(x(i))\n        part1=np.dot(alpha,y_matrix)\n        part2=sum(np.dot(part1[0,0],kernel[:,i]))\n        f_x_i=b+part2[0,0]\n        E[i]=f_x_i-y_matrix[i,0]\n        if (((y_matrix[i,0]*E[i])<-tol) & alpha[i]<C) | (((y_matrix[i,0]*E[i])>tol) & alpha[i]>0):\n            j=np.random.random_integers(0,890)\n            while(j==i):\n                j=np.random.random_integers(0,890)\n                \n            #computing f(x(j))\n            part1=np.dot(alpha,y_matrix)\n            part2=sum(np.dot(part1[0,0],kernel[:,j]))\n            f_x_j=b+part2[0,0]\n            E[j]=f_x_j-y_matrix[j,0]\n            \n            #saving old alphas\n            alpha_i_old=alpha[i]\n            alpha_j_old=alpha[j]\n            \n            #computing L & H\n            \n            if(y_matrix[i,0]==y_matrix[j,0]):\n                L=max(0,alpha[j]-alpha[i])\n                H=min(C,C+alpha[

In [28]:
from sklearn import svm

In [29]:
x_test=test_df[test_df.columns[2:]]
#x_test=test_df[traincolumn]
x_test_matrix=np.matrix(x_test)
test_df.columns
x_test_matrix.shape

(418L, 12L)

In [30]:
clf=svm.SVC()
clf.degree=3
clf.tol=1e-5
clf.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=1e-05, verbose=False)

In [31]:
y_test_matrix=clf.predict(x_test_matrix)

In [32]:
test_df['Survived']=y_test_matrix

In [33]:
test_df['Survived']=test_df['Survived'].apply(lambda x: (x+1)/2)

In [34]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": test_df["Survived"]
    })
submission.to_csv('titanic_svm.csv', index=False)

#Decision Tree

In [172]:
from sklearn import tree

In [173]:
model = tree.DecisionTreeClassifier()

In [174]:
y_train_dt=(y_train+1)/2
model.fit(x_train,y_train_dt)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')

In [175]:
y_test_matrix=model.predict(x_test_matrix)

In [179]:
test_df['Survived']=y_test_matrix
test_df['Survived']=test_df['Survived'].apply(lambda x: (x-1)*(-1))

In [180]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": test_df["Survived"]
    })
submission.to_csv('titanic_dt.csv', index=False)

#Neural Networks

In [237]:
import sklearn.neural_network as nn